Note: This notebook is inspired by: [GitHub Repository](https://github.com/aangelopoulos/conformal-prediction/blob/main/notebooks/meps-cqr.ipynb)

In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
!pip install -U --no-cache-dir gdown --pre

np.random.seed(42)

In [3]:
# Load cached data
if not os.path.exists('../data'):
    os.system('gdown 1h7S6N_Rx7gdfO3ZunzErZy6H7620EbZK -O ../data.tar.gz')
    os.system('tar -xf ../data.tar.gz -C ../')
    os.system('rm ../data.tar.gz')
    
data = np.load('../data/meps/meps-gbr.npz')
X, labels, upper, lower = data['X'], data['y'], data['upper'], data['lower']

In [4]:
# Problem setup
n=1000          # number of calibration points
alpha = 0.1     # 1-alpha is the desired coverage

In [5]:
# Split the softmax scores into calibration and validation sets (save the shuffling)
idx = np.array([1] * n + [0] * (labels.shape[0]-n)) > 0
np.random.shuffle(idx)
cal_labels, val_labels = labels[idx], labels[~idx]
cal_upper, val_upper = upper[idx], upper[~idx]
cal_lower, val_lower = lower[idx], lower[~idx]
val_X = X[~idx]

In [6]:
# Understanding the data
print("X.shape:      ", X.shape)
print("labels.shape: ", labels.shape)
print("upper.shape:  ", upper.shape)
print("lower.shape:  ", lower.shape)

print("upper:  ", upper)
print("labels: ", labels)
print("lower:  ", lower)

print("cal_upper:  ", cal_upper)
print("cal_labels: ", cal_labels)
print("cal_lower:  ", cal_lower)

'''
One of the row looks like:
0,53,1.0,25.93,58.47,3,15,21854.981705,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0
'''

X.shape:       (7893, 139)
labels.shape:  (7893,)
upper.shape:   (7893,)
lower.shape:   (7893,)
upper:   [2.78328316 2.62233302 2.72623802 ... 2.50649447 3.02326585 2.81011125]
labels:  [0.6931472 1.3862944 0.        ... 0.        2.6390574 1.3862944]
lower:   [0.         0.4394329  0.         ... 0.49617435 0.4394329  0.        ]
cal_upper:   [3.44945334 2.64417939 2.61650521 2.78397081 2.52990828 2.8977069
 3.32148928 2.3959731  2.58984756 3.48785698 2.56925327 2.57756018
 2.83218436 2.66859528 3.83746588 2.88119166 3.03102971 2.80311521
 3.04511388 2.6109199  2.81384602 3.33652572 2.89953835 2.6705085
 2.64367103 2.55588449 2.60324971 2.63170767 2.58502092 2.44399716
 2.5750515  2.72087651 2.77800786 4.22094354 3.00931685 2.63709743
 2.98550064 2.97681201 2.99762081 2.85614729 2.77720156 2.65069648
 2.8711461  2.69926745 3.35508746 3.1291836  2.77034039 2.62383137
 2.52315788 2.607473   3.58353871 2.59424659 2.83388982 2.80966175
 3.30609171 2.67566608 3.09731691 2.89948299 3.724746

'\nOne of the row looks like:\n0,53,1.0,25.93,58.47,3,15,21854.981705,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0\n'

### Conformal prediction

##### 1st time calibration

In [7]:
# Step 1: calculate the scores for the calibration set
cal_scores = np.maximum(cal_labels - cal_upper, cal_lower - cal_labels)

# Step 2: calculate the quantile
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(cal_scores, q_level, method='higher')
print(f'qhat: {qhat}')

# Step 3: calculate the prediction sets - for the validation set - by adjusting the initial values
prediction_sets = [val_lower - qhat, val_upper + qhat]

qhat: 0.4394328964201396


In [12]:
# Calculate empirical coverage of *calibration set* (before and after calibration)
cal_set_uncalibrated = [cal_lower, cal_upper]
cal_set_calibrated = [cal_lower - qhat, cal_upper - qhat]

print("For calibration set:")
empirical_coverage_original = ((cal_labels >= cal_set_uncalibrated[0]) & (cal_labels <= cal_set_uncalibrated[1])).mean()
print(f"The empirical coverage before calibration is: {empirical_coverage_original}")

empirical_coverage_calibrated = ((cal_labels >= cal_set_calibrated[0]) & (cal_labels <= cal_set_calibrated[1])).mean()
print(f"The empirical coverage after calibration is: {empirical_coverage_calibrated}")

For calibration set:
The empirical coverage before calibration is: 0.747
The empirical coverage after calibration is: 0.83


In [13]:
# Calculate empirical coverage of *validation set* (before and after calibration)
prediction_sets_uncalibrated = [val_lower, val_upper]

print("For validation set:")
empirical_coverage_uncalibrated = ((val_labels >= prediction_sets_uncalibrated[0]) & (val_labels <= prediction_sets_uncalibrated[1])).mean()
print(f"The empirical coverage before calibration is: {empirical_coverage_uncalibrated}")

empirical_coverage = ((val_labels >= prediction_sets[0]) & (val_labels <= prediction_sets[1])).mean()
print(f"The empirical coverage after calibration is: {empirical_coverage}")

For validation set:
The empirical coverage before calibration is: 0.7356738720441027
The empirical coverage after calibration is: 0.9316698099521253


##### 2nd time calibration
- this will result in qhat = 0 because cal_upper and cal_lower are updated to accomodate all the lables
- qhat = 0 means that the model is not calibrated, and the emperical coverage before and after calibration is the same

In [ ]:
cal_upper = cal_upper + qhat
cal_lower = cal_lower - qhat

# Step 1: calculate the scores for the calibration set
cal_scores = np.maximum(cal_labels - cal_upper, cal_lower - cal_labels)

# Step 2: calculate the quantile
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(cal_scores, q_level, method='higher')
print(f'qhat: {qhat}')